In [1]:
import numpy as np
import pickle
from torch.utils.data import DataLoader, Sampler
import random
import os
import fnmatch

import torch
import torch.nn as nn
from MLP import MultiLayerPerceptron

In [ ]:
from MyDataPreparationLSTM import CustomDataset

In [ ]:
def find_files(directory, pattern, maxdepth=None):
    flist = []
    for root, dirs, files in os.walk(directory):
        for basename in files:
            if fnmatch.fnmatch(basename, pattern):
                filename = os.path.join(root, basename)
                filename = filename.replace('\\\\', os.sep)
                if maxdepth is None:
                    flist.append(filename)
                else:
                    if filename.count(os.sep)-directory.count(os.sep) <= maxdepth:
                        flist.append(filename)
    return flist

In [ ]:
wind_files_pkl = find_files('/mnt/hippocamp/asavin/data/wind/wind_arrays_kara_norm', '*.pkl')
wind_files_pkl.sort()

In [ ]:
dataset = CustomDataset(wind_files_pkl, num_days=4, num_years=4)
dataset.select_random_years()

dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
pretrained_autoencoder_name = 'wind_pre_autoencoder_run002'

pretrained_encoder = torch.load(f'/app/Kara_plume_movement/wind/models/model_{pretrained_autoencoder_name}_encoder.pth', map_location=torch.device('cpu'))
lstm_network = nn.LSTM(pretrained_encoder.bottleneck, pretrained_encoder.bottleneck, num_layers=4, batch_first=True)
lstm_decoder = MultiLayerPerceptron(pretrained_encoder.bottleneck,
                                    output_size=pretrained_encoder.bottleneck,
                                    hidden_layers=[1024])
pretrained_decoder = torch.load(f'/app/Kara_plume_movement/wind/models/model_{pretrained_autoencoder_name}_decoder.pth', map_location=torch.device('cpu'))

In [ ]:
pretrained_encoder = pretrained_encoder.cuda()
lstm_network = lstm_network.cuda()
lstm_decoder = lstm_decoder.cuda()
pretrained_decoder = pretrained_decoder.cuda()

In [ ]:
for batch in dataloader:
    sequences, targets = batch
    print("Последовательность:", sequences.shape)
    print("Цель:", targets.shape)

In [ ]:
data, target = next(iter(dataloader))

In [ ]:
data_gpu = data.to(device='cuda', dtype=torch.float)

data_list = data_gpu.unbind(dim=1)
encoded_data_list = [pretrained_encoder.forward(t) for t in data_list]
decoded_target_list = [pretrained_decoder.forward(t) for t in encoded_data_list]

wind_vector = torch.stack(encoded_data_list, dim=1)
decoded_target = torch.stack(decoded_target_list, dim=1)
wind_vector.shape, decoded_target.shape

In [ ]:
encoded_data, (_, _) = lstm_network.forward(wind_vector)
split_tensors = encoded_data.unbind(dim=1)
processed_tensors = [pretrained_decoder.forward(lstm_decoder.forward(t)) for t in split_tensors]
decoded_data = torch.stack(processed_tensors, dim=1)
decoded_data.shape

In [ ]:
data_gpu = batch_data.to(device='cuda', dtype=torch.float)

data_list = data_gpu.unbind(dim=1)
encoded_data_list = [pretrained_encoder.forward(t) for t in data_list]
decoded_target_list = [pretrained_decoder.forward(t) for t in encoded_data_list]

wind_vector = torch.stack(encoded_data_list, dim=1)

encoded_data, (_, _) = lstm_network.forward(wind_vector)
split_tensors = encoded_data.unbind(dim=1)
processed_tensors = [pretrained_decoder.forward(lstm_decoder.forward(t)) for t in split_tensors]
decoded_data = torch.stack(processed_tensors, dim=1)

decoded_target = torch.stack(decoded_target_list, dim=1)

loss = loss_function(decoded_target, decoded_data)

In [ ]:
data.shape, target.shape

In [ ]:
len(dataset)

In [ ]:
len(dataset.indices)

In [ ]:
dataset.data.shape

In [ ]:
data_gpu = data.to(device='cuda', dtype=torch.float)
next_day_gpu = target.to(device='cuda', dtype=torch.float)

In [ ]:
type(data_gpu)

In [ ]:
pretrained_encoder_name = 'model_wind_pre_autoencoder_run001_encoder'
pretrained_encoder = torch.load(f'/app/Kara_plume_movement/wind/models/{pretrained_encoder_name}.pth', map_location=torch.device('cpu'))
lstm = nn.LSTM(pretrained_encoder.bottleneck, pretrained_encoder.bottleneck, num_layers=4, batch_first=True)
decoder = MultiLayerPerceptron(input_size=dataset.num_days * pretrained_encoder.bottleneck,
                               output_size=pretrained_encoder.bottleneck,
                               hidden_layers=[1024, 1024, 1024])

In [ ]:
pretrained_encoder = pretrained_encoder.cuda();
lstm = lstm.cuda();
decoder = decoder.cuda();

In [ ]:
pretrained_encoder.eval()
lstm.eval()
decoder.eval()

In [ ]:
encoded_data_list = []
for t in range(data_gpu.shape[1]):
    encoded_data_list.append(pretrained_encoder.forward(data_gpu[:,t,:,:,:]))
    
wind_vector = torch.stack(encoded_data_list, dim=1)
wind_vector.shape

In [ ]:
type(wind_vector)

In [ ]:
encoded_data, (_, _) = lstm.forward(wind_vector)
encoded_data.shape

In [ ]:
type(encoded_data)

In [ ]:
result_lstm = encoded_data.reshape(encoded_data.shape[0], -1)
result_lstm.shape

In [ ]:
decoded_data = decoder.forward(result_lstm)
decoded_data.shape

In [ ]:
encoded_target = pretrained_encoder.forward(next_day_gpu)
encoded_target.shape

In [ ]:
loss_function=torch.nn.MSELoss()

In [ ]:
loss = loss_function(encoded_target, decoded_data)
loss